In [1]:
import pandas as pd
import textwrap
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

In [2]:
_num_beams = 4
_no_repeat_ngram_size = 3
_length_penalty = 1
_min_length = 12
_max_length = 128
_early_stopping = True

In [3]:
df = pd.read_csv('bbc_text.csv')

In [4]:
doc = df[df.labels == 'business']['text'].sample(random_state=42)

In [5]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [6]:
input_text = doc.iloc[0]

In [7]:
print(input_text)

Christmas sales worst since 1981

UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.

Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.

The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.

The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The November-D

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
input_text = ' '.join(input_text.split())
input_tokenized = bart_tokenizer.encode(input_text, return_tensors='pt').to(device)
summary_ids = bart_model.generate(input_tokenized,
                                    num_beams=_num_beams,
                                    no_repeat_ngram_size=_no_repeat_ngram_size,
                                    length_penalty=_length_penalty,
                                    min_length=_min_length,
                                    max_length=_max_length,
                                    early_stopping=_early_stopping)

output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [9]:
output

['Christmas sales worst since 1981 UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981. Retail sales dropped by 1% on the month in December after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS. The last']